In [6]:
#https://mapping-api.herokuapp.com/exchange/NASDAQ
#https://mapping-api.herokuapp.com

In [2]:
import pandas as pd

List of companies and CIK and Exchange  
http://rankandfiled.com/#/data/tickers (downaloadable)

yet another ticker mappings https://dan.vonkohorn.com/2016/07/03/cik-ticker-mappings/

In [3]:
names = pd.read_csv('cik_ticker.csv', sep='|')

In [4]:
names.Exchange.unique()

array(['NYSE', nan, 'OTC', 'NASDAQ', 'OTCBB', 'NYSE ARCA', 'NYSE MKT',
       'BATS'], dtype=object)

In [5]:
NASDAQ = pd.read_json('https://mapping-api.herokuapp.com/exchange/NASDAQ')
len(NASDAQ)

2139

In [6]:
OTC = pd.read_json('https://mapping-api.herokuapp.com/exchange/OTC')
len(OTC)

2191

In [7]:
NYSE = pd.read_json('https://mapping-api.herokuapp.com/exchange/NYSE')
len(NYSE)

2736

In [8]:
df = pd.DataFrame()
df = df.append(NASDAQ)
df = df.append(OTC)
df = df.append(NYSE)

In [9]:
df.head()

,cik,exchange,irs,name,sic,ticker
0,1099290,NASDAQ,593404233,Sinocoking Coal & Coke Chemical Industries Inc,3312,AAC
1,6201,NASDAQ,751825172,American Airlines Group Inc,4512,AAL
2,8177,NASDAQ,581027114,Atlantic American Corp,6311,AAME
3,1158114,NASDAQ,760533927,Applied Optoelectronics Inc,3674,AAOI
4,320193,NASDAQ,942404110,Apple Inc,3571,AAPL


In [10]:
len(df)

7066

In [11]:
len(df.cik.unique())

7066

In [12]:
cik_codes = list(df.cik.unique())

In [13]:
cik_codes[:4]

[1099290, 6201, 8177, 1158114]

In [14]:
#edgar data shape /edgar/data/<CIK> / 0123456789-AB-CDEFGH.txt
#apple example https://www.sec.gov/Archives/edgar/data/320193/000032019319000119/0000320193-19-000119.txt

In [15]:
#browsig CIK
#https://www.sec.gov/cgi-bin/browse-edgar?CIK=1158114&owner=exclude&action=getcompany&Find=Search
#THIS ONE here is where the magic happens
#https://www.sec.gov/cgi-bin/browse-edgar?CIK=0000006201&type=10-K

In [52]:
#beautiful soup 
#https://www.digitalocean.com/community/tutorials/how-to-scrape-web-pages-with-beautiful-soup-and-python-3

---

In [19]:
import requests
from bs4 import BeautifulSoup
import re

In [2]:
page = requests.get('https://www.sec.gov/cgi-bin/browse-edgar?CIK=6201&type=10-K')

In [3]:
#object
soup = BeautifulSoup(page.text, 'html.parser')

In [4]:
## Pull all text from the table div
results_table = soup.find(class_='tableFile2')

In [5]:
# Pull text from all instances of <a> tag within BodyText div
results_items = results_table.find_all('a')

In [6]:
first_file = results_items[0].prettify()

In [7]:
first_file

'<a href="/Archives/edgar/data/6201/000000620119000009/0000006201-19-000009-index.htm" id="documentsbutton">\n Documents\n</a>\n'

In [8]:
matches=re.findall(r'\"(.+?)\"',first_file) #in python is r' '
#.+? is the "non-greedy" version of .+. It makes the regular expression match the smallest number of characters it can instead of the most characters it can

In [9]:
url_txt = 'https://www.sec.gov'+matches[0]
url_txt = url_txt.replace('-index.htm','.txt')

In [10]:
url_txt

'https://www.sec.gov/Archives/edgar/data/6201/000000620119000009/0000006201-19-000009.txt'

---
## Scrape Edgar

In [35]:
cik_codes[0:20]

[1099290, 6201, 8177, 1158114, 320193]

In [64]:
def get_url(query):
    page = requests.get(query)
    page_parsed = BeautifulSoup(page.text, 'html.parser')
    results_table = page_parsed.find(class_='tableFile2')
    return results_table.find_all('a')

In [ ]:
def get_txt_files(cik_codes):
    # INPUT = list of CIK codes
    docs_urls = []
    for cik in cik_codes:
        query = 'https://www.sec.gov/cgi-bin/browse-edgar?CIK='+str(cik)+'&type=10-K'
        results_files = get_url(query)
        #if there is no files in the registry means the table and list are empty
        if results_files != []:
            #print(str(query))
            results_items = results_files[0].prettify()  #here access the company
            raw_url = 'https://www.sec.gov'+re.findall(r'\"(.+?)\"',results_items)[0]
            url = raw_url.replace('-index.htm','.txt')
            docs_urls.append(url)
        else:
            docs_urls.append('no-url')
    return docs_urls

In [ ]:
import time; start_time = time.monotonic()
from datetime import datetime, timedelta
docs_urls = []
for code in cik_codes:
    query = 'https://www.sec.gov/cgi-bin/browse-edgar?CIK='+str(code)+'&type=10-K'
 #   print(query)#
    page = requests.get(query)
    page_parsed = BeautifulSoup(page.text, 'html.parser')
    results_table = page_parsed.find(class_='tableFile2')
    results_files = results_table.find_all('a')
    if results_files != []: #if there is no files in the registry means the table and list are empty
        #print(str(query))
        results_items = results_files[0].prettify()  #here access the company
        raw_url = re.findall(r'\"(.+?)\"',results_items)[0]
        url_txt = 'https://www.sec.gov'+raw_url
        url_doc = url_txt.replace('-index.htm','.txt')
        docs_urls.append(url_doc)
    else:
        docs_urls.append('no-url')

sttime = datetime.now().strftime('%Y%m%d_%H:%M_')
end_time = time.monotonic()
ex_time = timedelta(seconds=end_time - start_time)
print("Execution time: {}".format(ex_time))

In [41]:
#put runing time
#este duro como 40 mins

---

In [11]:
response = requests.get(url_txt)
data = response.text

In [19]:
data[:1000]

'<SEC-DOCUMENT>0000006201-19-000009.txt : 20190225\n<SEC-HEADER>0000006201-19-000009.hdr.sgml : 20190225\n<ACCEPTANCE-DATETIME>20190225073134\nACCESSION NUMBER:\t\t0000006201-19-000009\nCONFORMED SUBMISSION TYPE:\t10-K\nPUBLIC DOCUMENT COUNT:\t\t140\nCONFORMED PERIOD OF REPORT:\t20181231\nFILED AS OF DATE:\t\t20190225\nDATE AS OF CHANGE:\t\t20190225\n\nFILER:\n\n\tCOMPANY DATA:\t\n\t\tCOMPANY CONFORMED NAME:\t\t\tAmerican Airlines Group Inc.\n\t\tCENTRAL INDEX KEY:\t\t\t0000006201\n\t\tSTANDARD INDUSTRIAL CLASSIFICATION:\tAIR TRANSPORTATION, SCHEDULED [4512]\n\t\tIRS NUMBER:\t\t\t\t751825172\n\t\tSTATE OF INCORPORATION:\t\t\tDE\n\t\tFISCAL YEAR END:\t\t\t1231\n\n\tFILING VALUES:\n\t\tFORM TYPE:\t\t10-K\n\t\tSEC ACT:\t\t1934 Act\n\t\tSEC FILE NUMBER:\t001-08400\n\t\tFILM NUMBER:\t\t19628071\n\n\tBUSINESS ADDRESS:\t\n\t\tSTREET 1:\t\t4333 AMON CARTER BLVD\n\t\tCITY:\t\t\tFORT WORTH\n\t\tSTATE:\t\t\tTX\n\t\tZIP:\t\t\t76155\n\t\tBUSINESS PHONE:\t\t8179631234\n\n\tMAIL ADDRESS:\t\n\t\tSTREE

In [18]:
file = open('company.txt','w')
file.write(data)

30572408

In [21]:
def getReadableEntity(text):
    text = text.replace("\\n", "\n")
    text = text.replace("\\r\\n", " ")
    text = text.replace("\\xc3\\xbc", "u")
    text = text.replace("\\xc3\\xa4", "a")
    text = text.replace("\\xc2\\xa7", "")
    text = text.replace("\\xc3\\x9f", "ss")
    text = text.replace("\\xc3\\xb6", "o")
    text = text.replace("\\xc3\\x84", "A")
    text = text.replace("\\xc3\\x9c", "U")
    text = text.replace("\\xc3\\xa9", "e")
    text = text.replace("\\xc3\\xa8", "e")
    text = text.replace("\\xc3\\x96", "O")
    text = text.replace("\\xc3\\xa0", "a")

    text = text.replace("\r\n", " ")
    text = text.replace("\xc3\xbc", "u")
    text = text.replace("\xc3\xa4", "a")
    text = text.replace("\xc2\xa7", "")
    text = text.replace("\xc3\x9f", "ss")
    text = text.replace("\xc3\xb6", "o")
    text = text.replace("\xc3\x84", "A")
    text = text.replace("\xc3\x9c", "U")
    text = text.replace("\xc3\xa9", "e")
    text = text.replace("\xc3\xa8", "e")
    text = text.replace("\xc3\x96", "O")
    text = text.replace("\xc3\xa0", "a")
    return text

In [23]:
print(getReadableEntity(data[:1000]))

<SEC-DOCUMENT>0000006201-19-000009.txt : 20190225
<SEC-HEADER>0000006201-19-000009.hdr.sgml : 20190225
<ACCEPTANCE-DATETIME>20190225073134
ACCESSION NUMBER:		0000006201-19-000009
CONFORMED SUBMISSION TYPE:	10-K
PUBLIC DOCUMENT COUNT:		140
CONFORMED PERIOD OF REPORT:	20181231
FILED AS OF DATE:		20190225
DATE AS OF CHANGE:		20190225

FILER:

	COMPANY DATA:	
		COMPANY CONFORMED NAME:			American Airlines Group Inc.
		CENTRAL INDEX KEY:			0000006201
		STANDARD INDUSTRIAL CLASSIFICATION:	AIR TRANSPORTATION, SCHEDULED [4512]
		IRS NUMBER:				751825172
		STATE OF INCORPORATION:			DE
		FISCAL YEAR END:			1231

	FILING VALUES:
		FORM TYPE:		10-K
		SEC ACT:		1934 Act
		SEC FILE NUMBER:	001-08400
		FILM NUMBER:		19628071

	BUSINESS ADDRESS:	
		STREET 1:		4333 AMON CARTER BLVD
		CITY:			FORT WORTH
		STATE:			TX
		ZIP:			76155
		BUSINESS PHONE:		8179631234

	MAIL ADDRESS:	
		STREET 1:		4333 AMON CARTER BLVD
		CITY:			FORT WORTH
		STATE:			TX
		ZIP:			75261-9616

	FORMER COMPANY:	
		FORMER CONFORMED 

---
From EDGAR extraction implementation

In [13]:
# Text extraction patterns
mda_regex = r"item[^a-zA-Z\n]*\d\s*\.\s*management\'s discussion and analysis.*?^\s*item[^a-zA-Z\n]*\d\s*\.*"
qqd_regex = r"item[^a-zA-Z\n]*\d[a-z]?\.?\s*Quantitative and Qualitative Disclosures about " \
            r"Market Risk.*?^\s*item\s*\d\s*"
riskfactor_regex = r"item[^a-zA-Z\n]*\d[a-z]?\.?\s*Risk Factors.*?^\s*item\s*\d\s*"

In [16]:
re.findall(mda_regex, data, re.IGNORECASE | re.DOTALL | re.MULTILINE)

[]

In [17]:
data.find('Risk Factors')

60740